In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv
/kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


# **肥満リスクの多クラス予測**
(Multi-Class Prediction of Obesity Risk)

# 参考資料
**書籍**


**Kaggle**
- The fat CatBoost😼

**自分で作成したファイル**
- 銀行解約データセットを使用した二項分類
- 肝硬変の転帰の多クラス予測

**その他**


In [2]:
import numpy as np
import pandas as pd

FILE_PATH = '/kaggle/input/playground-series-s4e2/'
CREATE_DATA = '/kaggle/input/my-multi-class-prediction-of-obesity-risk'


# 確認 (data_import.py)
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    
    target_col = 'NObeyesdad'
    
    train = data[:20758].reset_index(drop=True)
    test = data[20758:].reset_index(drop=True).drop(target_col, axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop([target_col], axis=1) # 目的変数を指定する
    y = train[target_col].values # 目的変数を指定する

    return data,test,train,X,y

filename = 'data.csv'
data,test,train,X,y = file_to_xy(CREATE_DATA + '/' + filename)

読み込み完了 /kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


In [3]:
X = X.copy()
y = y.copy()

# 機械学習モデルの構築・学習・予測

## ライブラリのインポート

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 評価指標
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score

# 機械学習モデル
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# 実験
import datetime
import pickle


sample_sub = pd.read_csv(FILE_PATH + "sample_submission.csv")

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## データの分割

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## ハイパーパラメータ

In [6]:
lgb_params = {'lambda_l1': 1.3019828669773963, 
              'lambda_l2': 1.0378733814005926e-05, 
              'num_leaves': 43, 
              'feature_fraction': 0.5200508885969131, 
              'bagging_fraction': 0.8788385892118535, 
              'bagging_freq': 7, 
              'min_child_samples': 49}

xgb_params = {'max_depth': 7, 
              'min_child_weight': 5, 
              'learning_rate': 0.028015114445880215, 
              'n_estimators': 559, 
              'subsample': 0.9518356320628734, 
              'colsample_bytree': 0.46507972699185773, 
              'random_state': 42}

cat_params = {'iterations': 526,
    'depth': 3,
    'min_data_in_leaf': 13,
    'learning_rate': 0.0008029819794568197}



## モデルファイルと評価結果を出力する（ファイルの出力あり）, Main

In [7]:
def make_model_and_eval(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # accuracy
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    # F1_score
    f1_train = f1_score(y_train, y_pred_train, average='macro')
    f1_test = f1_score(y_test, y_pred_test, average='macro')
    # recall
    recall_train = recall_score(y_train, y_pred_train, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    # precision
    precision_train = precision_score(y_train, y_pred_train, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
#    # AUC
#     auc_train = roc_auc_score(y_train, y_pred_train)
#     auc_test = roc_auc_score(y_test, y_pred_test)
   # MAE
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
   # MSE
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
   # RMSE
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
   # r2_score
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

#     tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
#     tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    score_train = pd.DataFrame({'DataCategory':['Train'],
#                                'auc':[auc_train],
                               'accuracy':[acc_train],
                               'f1':[f1_train],
                               'recall':[recall_train],
                               'precision':[precision_train],
                               'MAE':[mae_train],
                               'MSE':[mse_train],
                               'RMSE':[rmse_train],
                               'r2':[r2_train],
#                                'tp':[tp_train],
#                                'fn':[fn_train],
#                                'fp':[fp_train],
#                                'tn':[tn_train]
                               })
    score_test = pd.DataFrame({'DataCategory':['Valid'],
#                               'auc':[auc_test],
                              'accuracy':[acc_test],
                              'f1':[f1_test],
                              'recall':[recall_test],
                              'precision':[precision_test],
                              'MAE':[mae_test],
                              'MSE':[mse_test],
                              'RMSE':[rmse_test],
                              'r2':[r2_test],
#                               'tp':[tp_test],
#                               'fn':[fn_test],
#                               'fp':[fp_test],
#                               'tn':[tn_test]
                              })
    score = pd.concat([score_train, score_test], ignore_index=True)
    
    importance = pd.DataFrame({'cols': X_train.columns,
                              'importance': model.feature_importances_})
    importance = importance.sort_values('importance', ascending=False)
    cols = pd.DataFrame({'X_cols': X_train.columns})
    display(score)
    
    return score, importance, model, cols

In [8]:
now = datetime.datetime.now().strftime("%Y%m%d%H")
target_output_dir = 'results_' + now

os.makedirs(target_output_dir, exist_ok=True)
print(target_output_dir)



models = {'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42), 
          'RandomForest':RandomForestClassifier(random_state=42),
          'GradientBoostingClassifier':GradientBoostingClassifier(random_state=42),
          'lightgbm': LGBMClassifier(**lgb_params),
          'xgboost': XGBClassifier(**xgb_params),
#           'BaggingClassifier': BaggingClassifier(random_state=42),
          'AdaBoostClassifier': AdaBoostClassifier(random_state=42),
#           'CatBoostClassifier': CatBoostClassifier(**cat_params),
         }

score_all = []
importance_all = []
for model_name, model in models.items():
    print(model_name)
    score, importance, model, cols = make_model_and_eval(model, X_train, X_valid, y_train, y_valid)
    score['model_name'] = model_name
    importance['model_name'] = model_name
    
    model_names = f'model_{model_name}.pickle'
    model_path = os.path.join(target_output_dir, model_names)
    with open(model_path, mode='wb') as f:
        pickle.dump(model, f, protocol=2)
    score_all.append(score)
    importance_all.append(importance)
score_all = pd.concat(score_all, ignore_index=True)
importance_all = pd.concat(importance_all, ignore_index=True)
cols = pd.DataFrame({'X_cols':X_train.columns})

score_name = 'score.csv'
importance_name = 'importance.csv'
cols_name = 'X_cols.csv'

score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)

score_all.to_csv(score_path, index=False)
importance_all.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)

results_2024020401
DecisionTreeClassifier


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
1,Valid,0.847303,0.831943,0.831859,0.832185,0.376204,1.277457,1.130246,0.649399


RandomForest


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.00000
1,Valid,0.893064,0.881411,0.880445,0.882909,0.271195,0.942678,0.970916,0.74128


GradientBoostingClassifier


...

,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.959472,0.955134,0.954834,0.955545,0.103216,0.359388,0.599490,0.899303
1,Valid,0.910886,0.901325,0.901067,0.901632,0.221821,0.755539,0.869218,0.792641


xgboost


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.959954,0.955448,0.955174,0.955867,0.098759,0.336866,0.580402,0.905614
1,Valid,0.907996,0.898087,0.897706,0.898589,0.235308,0.813825,0.902122,0.776644


AdaBoostClassifier


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.629411,0.567442,0.599230,0.621253,0.884500,2.879682,1.696963,0.193144
1,Valid,0.625482,0.558456,0.590388,0.609543,0.905588,2.980250,1.726340,0.182064


#### pickleファイルのインポート

In [9]:
import pickle

model_name = 'model_lightgbm.pickle'

model_path = os.path.join(target_output_dir, model_name)

with open(model_path, mode='rb') as f:
    model = pickle.load(f)
    
print(model)

LGBMClassifier(bagging_fraction=0.8788385892118535, bagging_freq=7,
               feature_fraction=0.5200508885969131,
               lambda_l1=1.3019828669773963, lambda_l2=1.0378733814005926e-05,
               min_child_samples=49, num_leaves=43)


## モデルの予測

In [10]:
target_col = 'NObeyesdad'
sample_sub[target_col] = model.predict(test)

[LightGBM] [Warning] feature_fraction is set=0.5200508885969131, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5200508885969131
[LightGBM] [Warning] lambda_l2 is set=1.0378733814005926e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.0378733814005926e-05
[LightGBM] [Warning] lambda_l1 is set=1.3019828669773963, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3019828669773963
[LightGBM] [Warning] bagging_fraction is set=0.8788385892118535, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8788385892118535
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


# 提出

In [11]:
inverse_mapping = {
                   0: 'Insufficient_Weight',
                   1: 'Normal_Weight', 
                   2: 'Obesity_Type_I',
                   3: 'Obesity_Type_II', 
                   4: 'Obesity_Type_III', 
                   5: 'Overweight_Level_I', 
                   6: 'Overweight_Level_II'
                  }

sample_sub[target_col] = sample_sub[target_col].map(inverse_mapping)
sample_sub.to_csv('submission.csv', index=False)
sample_sub.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


# 導入検討

### 標準化を行なったモデル

In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

clf1 = LogisticRegression(penalty='l2',
                         C=0.001,
                         solver='lbfgs',
                         random_state=1)
clf2 = DecisionTreeClassifier(max_depth=1,
                             criterion='entropy',
                             random_state=0)
clf3 = KNeighborsClassifier(n_neighbors=1,
                           p=2,
                           metric='minkowski')

pipe1 = Pipeline([['sc', StandardScaler()],
                 ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                 ['clf', clf3]])

clf_labels = ['Logistic regression', 'Decision tree', 'KNN']

print('10-fold cross validation:\n')

for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = cross_val_score(estimator=clf,
                            X=X_train,
                            y=y_train,
                            cv=10,
                            scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.64 (+/- 0.01) [Logistic regression]
Accuracy: 0.34 (+/- 0.00) [Decision tree]
Accuracy: 0.73 (+/- 0.01) [KNN]
